# Convert df_events to meta dataframe for pyphotometry analysis
- Meant to be a bridge between the pyphotometry analysis and df_events for the time being 

In [1]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
import seaborn as sns
import numpy as np
from trialexp.process.pycontrol.session_analysis import *
from trialexp.process.pycontrol.utils import *
from trialexp.process.pycontrol.session_analysis import Session
import sys
sys.path.append(r'../../../../pyControl')
import tools.data_import as di

        if you want to export datasets to perform Maching Learning tasks.
        To solve, type pip install sktime in your environment


## Read task definition

In [2]:
basefolder = r'C:\code\trialexp'
tasks = pd.read_csv(basefolder+'\\params\\tasks_params.csv', usecols=[1, 2, 3, 4], index_col=False)
task_name = 'reaching_go_spout_nov22'
conditions, triggers, events_to_process = Session.get_task_specs(tasks,  task_name)

## Read pycontrol file

In [24]:

pycontrol_dir = 'Z:\\Teris\\ASAP\\pycontrol\\reaching_go_spout_nov22\\'
filename = 'kms062-2023-02-06-112451'
smrx_output_dir = 'Z:\\Teris\\ASAP\\pycontrol\\reaching_go_spout_nov22\\processed\\'
# df_session = di.session_dataframe(pycontrol_dir+filename+'.txt')


filename = r'Z:\Julien\Data\head-fixed\pycontrol\reaching_go_spout_bar_nov22\kms063-2023-02-17-112445.txt'

df_session = di.session_dataframe(filename)
df_events = parse_session_dataframe(df_session)


Importing data file: kms063-2023-02-17-112445.txt


In [12]:
from datetime import datetime
session_time = datetime.strptime(df_events.attrs['Start date'], '%Y/%m/%d %H:%M:%S')

## Extract trial-related information from events

In [6]:
trial_window = [-2000, 4000]
timelim = [1000, 4000] # in ms

conditions, triggers, events_to_process = Session.get_task_specs(tasks, task_name)

df_events_trials, df_events = extract_trial_by_trigger(df_events, triggers[0], conditions+events_to_process, 
                                            trial_window, 'test', session_time)
df_events_trials.head()

ValueError: too many values to unpack (expected 2)

In [19]:
extract_trial_by_trigger(df_events, triggers[0], conditions+events_to_process, 
                                            trial_window, 'test', session_time)

,type,name,time,duration,value,trial_nb,trial_time,trigger


Compute conditions:

In [34]:
df_conditions = compute_conditions_by_trial(df_events_trials, conditions)
df_conditions.head()

,uid,trigger,valid,US_end_timer,water_on,Success:True,Success:False,busy_win,spout,button_press,hold_timer
trial_nb,,,,,,,,,,,
1.0,test_2023-02-06_11:24:51_1.0,busy_win,True,True,True,False,True,False,True,True,False
2.0,test_2023-02-06_11:24:51_2.0,busy_win,True,True,True,False,True,False,True,False,False
3.0,test_2023-02-06_11:24:51_3.0,busy_win,True,True,True,False,True,False,True,False,False
4.0,test_2023-02-06_11:24:51_4.0,busy_win,True,True,True,False,True,False,True,False,False
5.0,test_2023-02-06_11:24:51_5.0,busy_win,True,True,True,False,False,False,True,False,False


Compute success based on definition

In [6]:
session.create_metadata_dict(trial_window, timelim)

{'subject_ID': 62,
 'datetime': datetime.datetime(2023, 2, 6, 11, 24, 51),
 'timelim': [1000, 4000],
 'task': 'reaching_go_spout_nov22',
 'trial_window': [-2000, 4000],
 'com_port': 'COM3'}

In [26]:
compute_success(df_events_trials, df_conditions, task_name)

,uid,trigger,valid,US_end_timer,water_on,Success:True,Success:False,busy_win,spout,button_press,hold_timer,success
trial_nb,,,,,,,,,,,,
1.0,test_2023-02-06_11:24:51_1.0,busy_win,True,True,True,False,True,False,True,True,False,False
2.0,test_2023-02-06_11:24:51_2.0,busy_win,True,True,True,False,True,False,True,False,False,False
3.0,test_2023-02-06_11:24:51_3.0,busy_win,True,True,True,False,True,False,True,False,False,False
4.0,test_2023-02-06_11:24:51_4.0,busy_win,True,True,True,False,True,False,True,False,False,False
5.0,test_2023-02-06_11:24:51_5.0,busy_win,True,True,True,False,False,False,True,False,False,False
6.0,test_2023-02-06_11:24:51_6.0,busy_win,True,True,True,False,True,False,True,False,False,False
7.0,test_2023-02-06_11:24:51_7.0,busy_win,True,True,True,False,True,False,True,False,False,False
8.0,test_2023-02-06_11:24:51_8.0,busy_win,True,True,True,False,True,False,True,False,False,False
9.0,test_2023-02-06_11:24:51_9.0,busy_win,True,True,True,False,True,False,True,False,False,False


In [11]:
df_events_trials.loc[:,['spout_trial_time','US_end_timer_trial_time']]

,spout_trial_time,US_end_timer_trial_time
trial_nb,,
1.0,[-1.0],"[19.0, 21.0]"
2.0,"[-1.0, 26.0]",[20.0]
3.0,[0.0],[21.0]
4.0,"[0.0, 1124.0]","[21.0, 1145.0]"
5.0,"[0.0, 254.0, 271.0, 292.0, 307.0, 327.0, 344.0...","[21.0, 275.0, 365.0]"
6.0,"[-1567.0, 0.0, 561.0, 937.0, 1116.0]","[-1546.0, 21.0, 582.0, 958.0, 1137.0]"
7.0,"[-1995.0, -1782.0, -1474.0, 0.0]","[-1974.0, -1761.0, -1452.0, 21.0]"
8.0,"[-1.0, 458.0]","[21.0, 480.0]"
9.0,"[-1.0, 64.0]",[21.0]


In [21]:
reach_time_before_reward = df_events_trials.loc[:,['spout_trial_time','US_end_timer_trial_time']].apply(
        lambda x: find_last_time_before_list(x['spout_trial_time'], x['US_end_timer_trial_time']), axis=1)  
# select only trials with a spout event before a US_end_timer event
reach_bool = reach_time_before_reward.notnull()
# select trial where the hold time was present (not aborted)
reach_success_bool = reach_bool & df_conditions.
# set these trials as successful
print(reach_bool)  
df_conditions.loc[(reach_success_bool), 'success'] = True

trial_nb
1.0     True
2.0     True
3.0     True
4.0     True
5.0     True
6.0     True
7.0     True
8.0     True
9.0     True
10.0    True
11.0    True
12.0    True
13.0    True
14.0    True
15.0    True
16.0    True
17.0    True
18.0    True
19.0    True
20.0    True
dtype: bool
